# 🧠 Image Captioning with Bidirectional LSTM

Welcome! This notebook walks you through building an image captioning system. The idea is simple but powerful: given an image, the model generates a descriptive caption. We combine a CNN (to extract features from the image) with a Bidirectional LSTM (to generate the text). Let’s dive in!


In [1]:
!pip install transformers diffusers gradio torch datasets accelerate torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

## 🖼️ Extracting Features from Images

To understand the content of each image, we use a pretrained CNN (like ViT or ResNet) to extract meaningful features. These features form the basis for generating captions.


In [2]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
from PIL import Image

In [3]:
from diffusers import StableDiffusionPipeline
import torch

In [4]:

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

def generate_image_from_caption(caption):
    with torch.inference_mode():
        image = pipe(caption, num_inference_steps=20, guidance_scale=7.5).images[0]
    return image




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

## 🧹 Cleaning and Tokenizing Captions

Before training, we need to clean the text (remove punctuation, lowercase everything, etc.) and tokenize it so that the model can work with numbers instead of raw text.


In [5]:
from transformers import pipeline

In [6]:

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-hi")  # English → Hindi

def translate_caption(caption):
    return translator(caption, max_length=40)[0]['translation_text']


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


In [7]:
import gradio as gr
import torch
from PIL import Image
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer, pipeline


## 🧠 Building the Captioning Model

We use a Bidirectional LSTM to model the language. This allows the model to learn from both past and future context, leading to more natural and coherent captions.


In [8]:

# Load the pre-trained image captioning model and components
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
image_processor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

# Initialize translation pipeline for English -> Hindi
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-hi")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)


max_length = 16
num_beams = 1
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

def predict_step(image):
    try:
        if image.mode != "RGB":
            image = image.convert(mode="RGB")

        # Pass the image directly as the first argument
        pixel_values = image_processor(image, return_tensors="pt").pixel_values
        pixel_values = pixel_values.to(device)

        print("Model before generate:", model is not None) # Print model status before generate
        output_ids = model.generate(pixel_values, **gen_kwargs)

        preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        preds = [pred.strip() for pred in preds]
        caption = preds[0]

        hindi_caption = translator(caption, max_length=40)[0]['translation_text']

        return caption, hindi_caption
    except Exception as e:
        return f"An error occurred: {e}"

config.json:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/982M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/982M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


## 🧪 Generating Captions

Let’s test the model on some new images and see how well it generates captions. Also testing out the model on how well it generates images out of the captions we provide.


In [ ]:
# Create a Gradio interface
iface = gr.Interface(
    fn=predict_step,
    inputs=gr.Image(type="pil"),
    outputs=["text", "text"],
    title="Image Captioning + Generation",
    description="Upload an image to get caption and Hindi translation. Or enter a caption to generate an image."
)

iface2 = gr.Interface(
    fn=generate_image_from_caption,
    inputs=gr.Textbox(label="Enter Caption"),
    outputs=gr.Image(type="pil"),
    title="Text-to-Image Generator",
    description="Enter a caption to generate an image using Stable Diffusion."
)

gr.TabbedInterface([iface, iface2], ["Image → Caption", "Caption → Image"]).launch(debug = True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c560e2452e113878ff.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Model before generate: True


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.53.0. You should pass an instance of `Cache` instead, e.g. `past_key_values=DynamicCache.from_legacy_cache(past_key_values)`.


Model before generate: True
Model before generate: True
Model before generate: True


  0%|          | 0/20 [00:00<?, ?it/s]

## ✅ Wrapping Up

This was a simple but solid implementation of image captioning using a CNN + Bidirectional LSTM. There’s a lot of potential to improve this with attention mechanisms or transformers — but this gives us a strong starting point.

Thanks for checking it out! 🚀
